https://python.arviz.org/en/latest/api/generated/arviz.compare.html

In [1]:
from elisa.infer.fit import PosteriorResult

f_bay_result_Compt = PosteriorResult.load('/mnt/d/temp/Part10_posresul_Compt.pkl.xz', decompress='lzma')
f_bay_result_BlackbodyRad = PosteriorResult.load('/mnt/d/temp/Part10_posresul_BlackbodyRad.pkl.xz', decompress='lzma')
f_bay_result_OTTB = PosteriorResult.load('/mnt/d/temp/Part10_posresul_OTTB.pkl.xz', decompress='lzma')

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


In [2]:
import arviz as az
res_dic = {
    'Compt': f_bay_result_Compt.idata,
    'BlackbodyRad': f_bay_result_BlackbodyRad.idata,
    'OTTB': f_bay_result_OTTB.idata,
}
compare_res = az.compare(res_dic, ic='waic', var_name='channels', scale='deviance')
compare_res

/home/meotor/anaconda3/lib/python3.11/site-packages/arviz/stats/stats.py:1632: UserWarning: For one or more samples the posterior variance of the log predictive densities exceeds 0.4. This could be indication of WAIC starting to fail. 
See http://arxiv.org/abs/1507.04544 for details
  warnings.warn(
/home/meotor/anaconda3/lib/python3.11/site-packages/arviz/stats/stats.py:1632: UserWarning: For one or more samples the posterior variance of the log predictive densities exceeds 0.4. This could be indication of WAIC starting to fail. 
See http://arxiv.org/abs/1507.04544 for details
  warnings.warn(


,rank,elpd_waic,p_waic,elpd_diff,weight,se,dse,warning,scale
Compt,0,22.973980,2.378177,0.000000,9.347593e-01,7.476891,0.000000,True,deviance
BlackbodyRad,1,26.320752,1.988522,3.346772,6.528608e-15,5.782254,4.144575,False,deviance
OTTB,2,55.976882,4.658357,33.002901,6.524067e-02,10.242586,12.111212,True,deviance
